In [3]:
import matplotlib as plt
import numpy as np
import scipy
import pandas as pd
from sympy import *
import math
import qutip
import qutip as Q
import random

In [21]:
def ising_model_hamiltonian(L, J, g, periodic): # L = length of spin chain, J = overall constant, g = constant applied to field term
    # Identity and Pauli matrices for spin 1/2
    I = Q.qeye(2)
    z = Q.sigmaz()
    x = Q.sigmax()

    # create hamiltonian
    H = 0

    if periodic == 0:
#         print("The string of spins is not periodic")
    
        # Interaction term
        for i in range(L - 1):
            # Tensor product betweean spin i and i+1. Other positions are identity
            term = Q.tensor([I] * i + [z] + [z] + [I] * (L - i - 2))
            H += term
            #print("The interaction term is:")
            #print(term)

        # Transverse field term
        for i in range(L):
            # Pauli x matrix at each spin in the chain while the rest are identity
            hterm = Q.tensor([I] * i + [x] + [I] * (L - i - 1))
            #print("The transverse field term is:")
            #print(hterm)
            H += g * hterm
            
    elif periodic == 1:
#         print("The string of spins is periodic")
        for i in range(L - 1):
            # Tensor product betweean spin i and i+1. Other positions are identity
            term = Q.tensor([I] * i + [z] + [z] + [I] * (L - i - 2))
            H += term
            #print("The interaction term is:")
            #print(term)
            
        periodicterm = Q.tensor([I] * 0 + [z] + [I] * (L-2) + [z])
        H += periodicterm
        
        # Transverse field term
        for i in range(L):
            # Pauli x matrix at each spin in the chain while the rest are identity
            hterm = Q.tensor([I] * i + [x] + [I] * (L - i - 1))
            H += g * hterm
        
        
    H = -J * H
#     print("The Hamiltonian is:")
    return H

In [5]:
def svd(a):
    print("the original matrix is")
    print(a)
    b = np.linalg.eig(np.matmul(np.transpose(a),a))
    #c = np.linalg.eig(np.matmul(a,np.transpose(a)))
    eigenvalues, eigenvectors = b
    
    #order eigenvalues and vectors in descending order
    idx = eigenvalues.argsort()[::-1] 
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[:,idx]
    
    #set near zero terms equal to zero
    #print(eigenvalues)
    threshold1 = abs(eigenvalues) < 10**(-8)
    eigenvalues[threshold1] = 0
    
    #set near zero terms equal to zero
    #print(eigenvectors)
    threshold2 = abs(eigenvectors) < 10**(-8)
    eigenvectors[threshold2] = 0
    
    #trim zeroes from eigenvalues to make matrix sigma
    print("the eigenvalues are")
    print(eigenvalues)
    eigenvalstrimmed = np.trim_zeros(eigenvalues)
    print("the trimmed eigenvalues are")
    print(eigenvalstrimmed)
    
    #create matrix v
    print("the eigenvectors are")
    print(eigenvectors)
    v = np.transpose(eigenvectors)
    print("v is")
    print(v)
    
    #Find the singular values
    dimrow = np.count_nonzero(eigenvalstrimmed)
    dimcolumn = len(eigenvalues)
    sigma = np.zeros((dimrow,dimcolumn), dtype = float)
    svals = np.sqrt(eigenvalstrimmed)
    print("the singular values are")
    print(svals)
    
    #create matrix sigma
    np.fill_diagonal(sigma, svals)
    print("sigma is")
    print(sigma)
    print()
    
    #create matrix u
    u = np.zeros((a.shape[0], dimrow))
    for i in range(dimrow):
        u[:, i] = np.matmul(a/svals[i], eigenvectors[:, i])
    print("u is")
    print(u)
    print()
    
    #test if the SVD was correct
    originaltest = np.matmul(np.matmul(u,sigma), v)
    print("u*sigma*v gives")
    print(originaltest)
    print()
    
    #print the SVD
    print("the singular value decomposition for the matrix is")
    return u, sigma, v

In [6]:
def create_bipartition(vec, a_size): #input vector as well as number of spins wanted in a. Remainder will be in partition b
    length = len(vec)
    indexlist = list(range(length))
    
    random.shuffle(indexlist) #create random list to ensure random selection
    
    # Select a subset of the list
    a_index = indexlist[:a_size]
    
    # The remaining b list
    b_index = indexlist[a_size:]

    print('The index for a spins is')
    print(a_index)
    print('The index for b spins is')
    print(b_index)

In [7]:
def matrix_from_vector(spin_chain, vector, a_index, b_index):
    a_list = []
    for i in a_index:
        #print(i)
        a_list.append(vec[i][0])
        
    b_list = []
    for i in b_index:
        #print(i)
        b_list.append(vec[i][0])
    
    print('The list of a spins is')
    print(a_list)
    print('the list of b spins is')
    print(b_list)

In [20]:
def vec_to_matrix(spin_chain_length, vector, a_index, b_index):
    #function takes a length of spin chain, vector, and the index of particles in a or b
    #returns corresponding mapping to a matrix for the vector
    #only works for spin chain length = 3
    length = 2**spin_chain_length
    marray = []
    for i in range(length):
        binary = bin(i)[2:]
        long_binary = binary.zfill(spin_chain_length)
        array = [int(digit) for digit in long_binary]
        marray.append(array)
    #print(marray)
    
    if len(a_index) == 1: #if there is one spin in a
        a_positions = []
        for i in range(len(a_index)):
            pos = a_index[i]
            for j in range(len(marray)):
                a_positions.append(marray[j][pos])
                marray[j].pop(pos)
                b_positions = marray
        #print(a_positions)
        #print(b_positions)

        b_cols = []
    
        for i in b_positions:
            # Convert the list of 1s and 0s to a binary string
            binary_string = ''.join(map(str, i))
            # Convert the binary string to an integer
            number = int(binary_string, 2)
            b_cols.append(number)
    
#  

        matrix = np.zeros((max(a_positions)+1, max(b_cols)+1), dtype=float)
    
        for value, row, col in zip(vector, a_positions, b_cols):
            matrix[row, col] = float(value[0])
        
#         print("The matrix is")
        return(matrix)
        
    
    elif len(b_index) == 1: #if there is one spin in b
        b_positions = []
        for i in range(len(b_index)):
            pos = b_index[i]
            for j in range(len(marray)):
                b_positions.append(marray[j][pos])
                marray[j].pop(pos)
                a_positions = marray
        #print(a_positions)
        #print(b_positions)

        a_rows = []
    
        for i in a_positions:
            # Convert the list of 1s and 0s to a binary string
            binary_string = ''.join(map(str, i))
            # Convert the binary string to an integer
            number = int(binary_string, 2)
            a_rows.append(number)
    
#         print("The corresponding rows are")
#         print(a_rows)
#         print("The corresponding columns are")
#         print(b_positions)
    
        matrix = np.zeros((max(a_rows)+1, max(b_positions)+1), dtype=float)
    
        for value, row, col in zip(vector, a_rows, b_positions):
            matrix[row, col] = float(value[0])
        
#         print("The matrix is")
        return matrix
